In [ ]:
import pandas as pd
import matplotlib
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
%run data.ipynb
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df_217_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24217_3p_6mG.xlsx')
df_217_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24217_5p_6mG.xlsx')
df_218_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24218_3p.xlsx')
df_218_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24218_5p.xlsx')
df_203_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24203_3p_6mG.xlsx')
df_203_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24203_5p_6mG.xlsx')
df_232_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24232_3p.xlsx')
df_232_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24232_5p.xlsx')
df_204_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24204_3p.xlsx')
df_204_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24204_5p.xlsx')
df_189_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24189_3p.xlsx')
df_189_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24189_5p.xlsx')

In [ ]:
df_templates_5p = [df_217_5p, df_203_5p, df_204_5p, df_189_5p, df_218_5p, df_232_5p]
df_templates_3p = [df_217_3p, df_203_3p, df_204_3p, df_189_3p, df_218_3p, df_232_3p]

In [ ]:
df0_ctrl = load_uc_glu(1)
df0_ctrl_alkb = load_uc_glu(2)
plotly_zone(df0_ctrl)

df1_ctrl = load_uc2(5, intact=True)
df1_ctrl_alkb = load_uc2(6, intact=True)
plotly_zone(df1_ctrl)

In [ ]:
df0 = pd.read_excel('/Users/bryan/Documents/Data/UC/UC_glu_5_rosa.xlsx')
df0 = thermo_df(df0)

df1 = load_uc2(17)

df2 = pd.read_excel('/Users/bryan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Glu_FA_17.xlsx')
df2 = thermo_df(df2)

In [ ]:
dfm_5ps = list()
dfm_3ps = list()
dfm_3ps_rev = list()
df_sample = df1.copy()
for df_template in df_templates_5p:
    dfm = match_dfs(df_template, df_sample)
    dfm_5ps.append(dfm)

for df_template in df_templates_3p:
    dfm = match_dfs(df_template, df_sample)
    dfm_3ps.append(dfm)
    
    dfm_rev = dfm.copy()
    dfm_rev.Mass = df_template.Mass.max() + H2O - dfm_rev.Mass
    dfm_rev = dfm_rev[dfm_rev.Mass>200]
    dfm_3ps_rev.append(dfm_rev)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 5), 
                               sharex=True, sharey=True, 
                               gridspec_kw={'height_ratios': [5, 1]})

dfs = [
       (dfm_5ps[0], "24160 5'"),
       (dfm_5ps[1], "24159 5'"),
       (dfm_5ps[2], "24174 5'"),
       (dfm_5ps[3], "24175 5'"),
#        (dfm_5ps[4], "23947 5'"),
       
       (dfm_3ps_rev[0], "24160 3' --> 5'"),
       (dfm_3ps_rev[1], "24159 3' --> 5'"),
       (dfm_3ps_rev[2], "24174 3' --> 5'"),
       (dfm_3ps_rev[3], "24175 3' --> 5'"),
#        (dfm_3ps_rev[4], "23947 3' --> 5'"),
      ] 

total = len(dfs)
df_isoforms = list()
for idx, item in enumerate(dfs):
    df_iso = item[0].copy()
    print(idx, df_iso.shape)
    if 'Position' in df_iso.columns:
        df_iso['pos'] = df_iso.Position
    else:
        df_iso['pos'] = df_iso.Mass/320
        df_iso.pos.astype(int)
    label = item[1]
    ax1.scatter(df_iso.Mass, [total - idx]*len(df_iso.Mass), label=label)
    
    df_iso['Isoform'] = idx + 1
    df_isoforms.append(df_iso)
# plt.legend(loc=3)
ax1.xaxis.set_ticks(np.arange(0, 25000, 1000))

df_merge_5p = pd.concat(df_isoforms).drop_duplicates(subset=['Mass', 'RT'])
print(df_merge_5p.shape)
ax2.scatter(df_merge_5p.Mass, [3]*df_merge_5p.shape[0], color='b')

ax = plt.gca()
plt.draw()
ax2.set_xticklabels(ax2.get_xticks(), rotation = 45)
# df_merge_5p = pd.concat(df_isoforms)
# output(df_merge_5p, plt, fig, 1, 'phe_complement_5p3p')

# fig.tight_layout()
# plt.savefig('/Users/bryan/Downloads/tRNA_phe_olis2c1_complement.svg', transparent=True, dpi=300)

In [ ]:
df_masses = pd.read_excel('~/Downloads/glu_isoforms.xlsx')
df_hm = match_dfs(df_masses, df0_ctrl)
df_hm_alkb = match_dfs(df_masses, df0_ctrl_alkb)
df_hm['Ratio'] = df_hm.Vol / df_hm.Vol.max()
df_hm_alkb['Ratio'] = df_hm_alkb.Vol / df_hm.Vol.max()
df_hm, df_hm_alkb

In [ ]:
plotly_zones(df_hm, df_hm_alkb, y='Ratio')

In [ ]:
fig = plt.figure(figsize=(10, 6))
plt.scatter(df_hm.Mass, df_hm.Ratio, s=100, color='black', label='Wild-type') ##bdc1c1
plt.scatter(df_hm_alkb.Mass, df_hm_alkb.Ratio, s=100, color='r', label='AlkB') ##d9d9d9

for idx, row in df_hm.iterrows():
        x_pos = row.Mass
        y_pos = row.Ratio
        mass = '{:.4f}'.format(x_pos)
        plt.annotate(text=mass, size=13, xy=(x_pos, y_pos), 
                     textcoords="offset points", xytext=(10, -20), ha='center')

for idx, row in df_hm_alkb.iterrows():
        x_pos = row.Mass
        y_pos = row.Ratio
        mass = '{:.4f}'.format(x_pos)
        plt.annotate(text=mass, size=13, xy=(x_pos, y_pos), 
                     textcoords="offset points", xytext=(10, -20), ha='center', color='r')

plt.legend()

fig.tight_layout()
plt.savefig('/Users/bryan/Downloads/glu_homology_v2.svg', transparent=True, dpi=300)